In [16]:
import sklearn
import pandas as pd 
import numpy as np 
import os 
import re
import sys 
import ast 

In [17]:
results_dir = "../../../european-city-data/rag-sustainability/results/results-combined_prompts/"
folders = os.listdir(results_dir)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# can we just stick to Tf-Idf to vectorize, or do we want to use embeddings - static (Word2Vec) or contextual (BERT)? 

def compute_similarity(paths):

    documents = [open(f).read() for f in paths]
    tfidf = TfidfVectorizer().fit_transform(documents)
    # no need to normalize, since Vectorizer will return normalized tf-idf
    pairwise_similarity = tfidf * tfidf.T

    return round(pairwise_similarity[0,1], 3)

# corpus = ["I'd like an apple", "An apple a day keeps the doctor away",]
# tfidf = TfidfVectorizer().fit_transform(corpus)
#     # no need to normalize, since Vectorizer will return normalized tf-idf
# pairwise_similarity = tfidf * tfidf.T
# print(round(pairwise_similarity[0,1], 3))

In [19]:
sim_results = []
for model in folders:
    if ".csv" in model: 
        continue 

    for prompt in os.listdir(os.path.join(results_dir, model)):

        non_sus_paths = [os.path.join(results_dir, model, prompt, "response.txt"), os.path.join(results_dir, model, prompt, "context.txt")]
        sus_paths = [os.path.join(results_dir, model, prompt, "response_sustainable.txt"), os.path.join(results_dir, model, prompt, "context_sustainable.txt")]

        sim_score = compute_similarity(non_sus_paths)
        sim_score_sustainable = compute_similarity(sus_paths)

        sim_results.append({
            'model': model, 
            'prompt_id': prompt,
            'context_response_sim': sim_score, 
            'context_response_sim_sustainable': sim_score_sustainable,
        })

In [21]:
sim_results_df = pd.DataFrame(sim_results)

In [22]:
sim_results_df.head()

,model,prompt_id,context_response_sim,context_response_sim_sustainable
0,llama3point1-instruct,prompt_17_gemini-ui,0.751,0.675
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,0.601,0.543
2,llama3point1-instruct,prompt_42_gpt-4o-mini,0.419,0.777
3,llama3point1-instruct,prompt_0_gpt-4o-mini,0.647,0.605
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,0.663,0.576


In [23]:
sim_results_df.to_csv(f"{results_dir}/context_response_similarity_scores.csv")